In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

links = []
dates = []

start_date = pd.Timestamp(2024, 5, 23).date()
end_date = pd.Timestamp(2024, 5, 24).date()

current_date = start_date
while current_date < end_date:

    url = f"https://tribune.com.pk/listing/{current_date.strftime('%Y-%m-%d')}"

    response = requests.get(url)

    if response.status_code == 200:

        soup = bs(response.text, 'html.parser')

        news_elements = soup.find_all("div", class_="horiz-news3-caption")

        for news_element in news_elements:
            link_element = news_element.find("a", href=True)
            if link_element:
                link = link_element['href']
                
                date_element = news_element.find("div", class_="morestories-author").find("span")
                date_str = date_element.text.strip().split('|')[1].strip().split(' ')[1:]
                date_str = ' '.join(date_str)
                date = pd.to_datetime(date_str, format="%b %d, %Y").date()

                links.append(link)
                dates.append(date)

    # Move to the next date
    current_date += pd.Timedelta(days=1)


df = pd.DataFrame({'links': links, 'dates': dates})

with pd.ExcelWriter('final_tribune_links.xlsx') as writer:
    df.to_excel(writer, sheet_name='news', index=False)


In [8]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

import pandas as pd

#Read the DataFrame containing links
news_links = pd.read_excel('C:/Users/azain/final_tribune_links.xlsx')

#Convert 'dates' column to datetime and remove the time component
news_links['dates'] = pd.to_datetime(news_links['dates']).dt.date

#Continue with your existing code...
#Function to extract articles from a given URL
def extract_paragraphs(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = bs(response.content, 'html.parser')
        articles = soup.find_all('p')
        return ' '.join(p.get_text(strip=True) for p in articles)
    else:
        print(f"Failed to fetch URL: {url}")
        return None

#Create an empty list to store articles
all_paragraphs = []

#Iterate over each row in the DataFrame and extract articles
for index, row in news_links.iterrows():
    print(f"Extracting articles from {row['links']} ...")
    articles = extract_paragraphs(row['links'])
    all_paragraphs.append(articles)

#Add the articles to the DataFrame
news_links['articles'] = all_paragraphs

#Write to Excel
with pd.ExcelWriter('final_tribune_articles.xlsx') as writer:
    news_links.to_excel(writer, index=False)

Extracting articles from https://tribune.com.pk/story/2467959/not-seeking-loans-but-joint-collaboration-says-pm-shehbaz ...
